# Healthcare IoT Device Network Intrusion Detection System

### Creating the dataframes

In [1]:
import pandas as pd
import os
import glob

# options to see all columns in dataframes
pd.set_option("display.max_columns", None)

# get directory folder of train and test csv files (make sure to replace with path on your computer)
train_dir = r"C:\Users\spark\CMPE 132 Project\CMPE132Project\Dataset\Train".replace(
    "\\", "/"
)
test_dir = r"C:\Users\spark\CMPE 132 Project\CMPE132Project\Dataset\Test".replace(
    "\\", "/"
)

# store paths of all csv files in the folder in a list
train_csv_files = glob.glob(os.path.join(train_dir, "*.csv"))
test_csv_files = glob.glob(os.path.join(test_dir, "*.csv"))

# list to store csv files as dataframes
train_dataframes = []
test_dataframes = []

# create dataframes for each csv with column to label the class type
# get filename from path
for file in train_csv_files:
    filename = os.path.basename(file)

    # split the file name to get the class of the attack
    attack_type = filename.split("_train.pcap.csv")[0]

    # since some attacks are split into multiple files we need to remove numbers from the string
    # if last character is a number we remove it to get the name of the attack type
    if attack_type[-1].isdigit():
        attack_type = attack_type[:-1]

    # create a dataframe for the specific csv file
    df = pd.read_csv(file)

    # create a new column to classify the attack type in that csv file
    df["classification"] = attack_type

    # append the dataframe to the dataframes list
    train_dataframes.append(df)

    # delete the dataframe
    del df

# do the same for the test data
for file in test_csv_files:
    filename = os.path.basename(file)

    # split the file name to get the class of the attack
    attack_type = filename.split("_test.pcap.csv")[0]

    # since some attacks are split into multiple files we need to remove numbers from the string
    # if last character is a number we remove it to get the name of the attack type
    if attack_type[-1].isdigit():
        attack_type = attack_type[:-1]

    # create a dataframe for the specific csv file
    df = pd.read_csv(file)

    # create a new column to classify the attack type in that csv file
    df["classification"] = attack_type

    # append the dataframe to the dataframes list
    test_dataframes.append(df)

    # delete the dataframe
    del df


# combine all of the dataframes into one
train_df = pd.concat(train_dataframes, ignore_index=True)
test_df = pd.concat(test_dataframes, ignore_index=True)

# downsample to a certain fraction of random samples per class depending on how many rows in the class
# if smaller class don't cut out rows or if normal traffic don't cut out
sampled_dfs = []
target_size = 50000

# get ratio of each class
class_distribution = train_df["classification"].value_counts(normalize=True)
samples_per_class = (class_distribution * target_size).round().astype(int)

for class_name, n_samples in samples_per_class.items():
    class_subset = train_df[train_df["classification"] == class_name]
    sampled_df = class_subset.sample(n=n_samples, random_state=42)
    sampled_dfs.append(sampled_df)
train_df = pd.concat(sampled_dfs, ignore_index=True)

# add binary label to the training and testing dataframe in order to test binary classification as well
train_df["binary_classification"] = train_df["classification"].apply(
    lambda binary: 0 if binary.lower() == "benign" else 1
)
test_df["binary_classification"] = test_df["classification"].apply(
    lambda binary: 0 if binary.lower() == "benign" else 1
)

print(train_df["classification"].value_counts())
print(train_df["binary_classification"].value_counts())
print(test_df["binary_classification"].value_counts())

print(f"rows in training dataset: {train_df.shape[0]}")
print(f"rows in testing dataset: {test_df.shape[0]}")

classification
TCP_IP-DDoS-UDP            11423
TCP_IP-DDoS-ICMP           10735
TCP_IP-DDoS-TCP             5617
TCP_IP-DDoS-SYN             5600
TCP_IP-DoS-UDP              3959
TCP_IP-DoS-SYN              3086
TCP_IP-DoS-ICMP             2907
TCP_IP-DoS-TCP              2656
Benign                      1346
MQTT-DDoS-Connect_Flood     1208
Recon-Port_Scan              586
MQTT-DoS-Publish_Flood       310
MQTT-DDoS-Publish_Flood      193
Recon-OS_Scan                118
ARP_Spoofing                 112
MQTT-DoS-Connect_Flood        89
MQTT-Malformed_Data           36
Recon-VulScan                 15
Recon-Ping_Sweep               5
Name: count, dtype: int64
binary_classification
1    48655
0     1346
Name: count, dtype: int64
binary_classification
1    1576575
0      37607
Name: count, dtype: int64
rows in training dataset: 50001
rows in testing dataset: 1614182


### Data Preprocessing

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# create training and testing data from the dataframes made earlier
# must drop the classification rows in order to work with the feature data
# create both y training and testing data for multi class classification and binary classification
X_train = train_df.drop(columns=["classification", "binary_classification"])
y_train_multi = train_df["classification"]
y_train_binary = train_df["binary_classification"]

X_test = test_df.drop(columns=["classification", "binary_classification"])
y_test_multi = test_df["classification"]
y_test_binary = test_df["binary_classification"]

# standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# use PCA to reduce feature
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Reduced number of features after PCA: {X_train_pca.shape[1]}")

del X_train, X_test, X_train_scaled, X_test_scaled

Reduced number of features after PCA: 22


### Random Forest Classifier Training

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

In [4]:
# train random forest classifier model
# create two different RF classifiers, one to fit the multi class classification data
# and one to fit the binary classification data
rf_multi = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_binary = RandomForestClassifier(random_state=42, n_jobs=-1)

rf_multi.fit(X_train_pca, y_train_multi)
rf_binary.fit(X_train_pca, y_train_binary)

RandomForestClassifier(n_jobs=-1, random_state=42)

#### Accuracy of Random Forest Classifier Model

In [5]:
# check accuracy of model
rf_y_pred_multi = rf_multi.predict(X_test_pca)
rf_y_pred_binary = rf_binary.predict(X_test_pca)

# classification reports
rf_multi_classification_report = classification_report(
    y_test_multi, rf_y_pred_multi, output_dict=True
)
rf_binary_classification_report = classification_report(
    y_test_binary, rf_y_pred_binary, output_dict=True
)

print(
    "\nClassification Report for Multi-Class Classification:\n",
    rf_multi_classification_report,
)
print(
    "\nClassification Report for Binary Classification:\n",
    rf_binary_classification_report,
)


Classification Report for Multi-Class Classification:
 {'ARP_Spoofing': {'precision': 0.3142016188373804, 'recall': 0.4896788990825688, 'f1-score': 0.3827879874495742, 'support': 1744.0}, 'Benign': {'precision': 0.8891977163461539, 'recall': 0.9442656952163161, 'f1-score': 0.9159047238306488, 'support': 37607.0}, 'MQTT-DDoS-Connect_Flood': {'precision': 0.9898075013006669, 'recall': 0.9985447084645481, 'f1-score': 0.9941569083869742, 'support': 41916.0}, 'MQTT-DDoS-Publish_Flood': {'precision': 0.9921568627450981, 'recall': 0.15030893536121673, 'f1-score': 0.26106696935300794, 'support': 8416.0}, 'MQTT-DoS-Connect_Flood': {'precision': 0.9817415730337079, 'recall': 0.8930054295752156, 'f1-score': 0.9352734570998494, 'support': 3131.0}, 'MQTT-DoS-Publish_Flood': {'precision': 0.5434490696336083, 'recall': 0.9992945326278659, 'f1-score': 0.7040258449304175, 'support': 8505.0}, 'MQTT-Malformed_Data': {'precision': 0.8606271777003485, 'recall': 0.28277046365197483, 'f1-score': 0.425678586

#### Optimizing the Random Forest Classifier

In [6]:
# parameters to search through
rf_params_grid = {
    "n_estimators": [100, 200, 400, 800, 1000],
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": [None, 10, 20, 40, 80, 100],
    "min_samples_split": [2, 4, 8, 10, 12],
    "min_samples_leaf": [1, 2, 4, 8, 10],
    "max_features": ["sqrt", "log2", None],
}

# initialize RandomizedSearchCV object for both multi class and binary classification
rf_multi_randomized_search = RandomizedSearchCV(
    estimator=rf_multi,
    param_distributions=rf_params_grid,
    random_state=42,
    n_iter=20,
    n_jobs=-1,
    verbose=2,
    scoring="f1_macro",
)
rf_binary_randomized_search = RandomizedSearchCV(
    estimator=rf_binary,
    param_distributions=rf_params_grid,
    n_iter=20,
    random_state=42,
    n_jobs=-1,
    verbose=2,
    scoring="f1",
)

# start search on hyper parameters for both binary and multi class classification
rf_multi_search = rf_multi_randomized_search.fit(X_train_pca, y_train_multi)
rf_binary_search = rf_binary_randomized_search.fit(X_train_pca, y_train_binary)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [7]:
# print best estimator and best parameters
print(
    f"Best estimator for multi class classification: {rf_multi_search.best_estimator_}"
)
print(f"Best parameters for multi class classification: {rf_multi_search.best_params_}")
print(f"Best estimator for binary classification: {rf_binary_search.best_estimator_}")
print(f"Best parameters for binary classification: {rf_binary_search.best_params_}")

# print classification report for those parameters for both multi class and binary classification
rf_y_pred_multi_random_search = rf_multi_search.predict(X_test_pca)
rf_y_pred_binary_random_search = rf_binary_search.predict(X_test_pca)

# classification reports
rf_multi_search_classification_report = classification_report(
    y_test_multi, rf_y_pred_multi_random_search, output_dict=True
)
rf_binary_search_classification_report = classification_report(
    y_test_binary, rf_y_pred_binary_random_search, output_dict=True
)

print(
    "\nClassification Report for RandomizedSearchCV Multi Class Classification:\n",
    rf_multi_search_classification_report,
)
print(
    "\nClassification Report for RandomizedSearchCV Binary Classification:\n",
    rf_binary_search_classification_report,
)

Best estimator for multi class classification: RandomForestClassifier(max_features='log2', min_samples_split=4, n_jobs=-1,
                       random_state=42)
Best parameters for multi class classification: {'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'criterion': 'gini'}
Best estimator for binary classification: RandomForestClassifier(criterion='entropy', max_depth=40, min_samples_leaf=4,
                       min_samples_split=10, n_estimators=1000, n_jobs=-1,
                       random_state=42)
Best parameters for binary classification: {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 40, 'criterion': 'entropy'}

Classification Report for RandomizedSearchCV Multi Class Classification:
 {'ARP_Spoofing': {'precision': 0.30517241379310345, 'recall': 0.507454128440367, 'f1-score': 0.38113695090439276, 'support': 1744.0}, 'Benign': {'precision': 0.8885455

#### Tables for Model Scores Before and After Optimization

In [ ]:
def classification_report_tables(
    report_before,
    report_after,
    algorithm_name="Model",
    classification_type="Multiclass",
):
    metrics = ["precision", "recall", "f1-score"]

    # get the classification labels
    class_labels = [
        k
        for k in report_before
        if isinstance(report_before[k], dict) and k not in ["macro avg", "weighted avg"]
    ]

    # create table 1 for scores for each class
    class_rows = []
    for label in class_labels:
        row = {"Label": label}
        for metric in metrics:
            row[f"{metric.capitalize()} (Before)"] = report_before[label].get(metric)
            row[f"{metric.capitalize()} (After)"] = report_after[label].get(metric)
        row["Support"] = report_after[label].get("support")
        class_rows.append(row)

    df_class = pd.DataFrame(class_rows).set_index("Label")

    # add support column
    cols = df_class.columns.tolist()
    if "F1-score (After)" in cols and "Support" in cols:
        idx = cols.index("F1-score (After)")
        cols.insert(idx + 1, cols.pop(cols.index("Support")))
        df_class = df_class[cols]

    # create table for macro avg and weighted avg scores
    avg_types = ["macro avg", "weighted avg"]
    avg_rows = []
    for avg in avg_types:
        row = {"Average Type": avg}
        for metric in metrics:
            row[f"{metric.capitalize()} (Before)"] = report_before[avg].get(metric)
            row[f"{metric.capitalize()} (After)"] = report_after[avg].get(metric)
        avg_rows.append(row)
    df_avg = pd.DataFrame(avg_rows).set_index("Average Type")

    # create table for accuracy
    df_acc = pd.DataFrame(
        {
            "Accuracy (Before)": [report_before.get("accuracy")],
            "Accuracy (After)": [report_after.get("accuracy")],
        },
        index=["Accuracy"],
    )

    # title of table
    title_prefix = f"{algorithm_name} ({classification_type})"

    # create gradient for all columns except the support column
    gradient_cols = [col for col in df_class.columns if col != "Support"]

    # format of values in table
    format_dict = {col: "{:.3f}" for col in df_class.columns}
    format_dict["Support"] = "{:.1f}"

    # display the tables
    display(
        df_class.style.set_caption(f"{title_prefix} Per-Class Metrics")
        .background_gradient(cmap="YlGnBu", axis=None, subset=gradient_cols)
        .format(format_dict)
    )

    display(
        df_avg.style.set_caption(f"{title_prefix} Average Metrics")
        .background_gradient(cmap="YlGnBu", axis=None)
        .format("{:.3f}")
    )

    display(
        df_acc.style.set_caption(f"{title_prefix} Overall Accuracy")
        .background_gradient(cmap="YlGnBu", axis=None)
        .format("{:.3f}")
    )

In [9]:
classification_report_tables(
    rf_multi_classification_report,
    rf_multi_search_classification_report,
    algorithm_name="Random Forest Classifier",
    classification_type="Multiclass Classification",
)

classification_report_tables(
    rf_binary_classification_report,
    rf_binary_search_classification_report,
    algorithm_name="Random Forest Classifier",
    classification_type="Binary Classification",
)


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After),Support
Label,,,,,,,
ARP_Spoofing,0.314,0.305,0.490,0.507,0.383,0.381,1744.0
Benign,0.889,0.889,0.944,0.940,0.916,0.914,37607.0
MQTT-DDoS-Connect_Flood,0.990,0.991,0.999,0.998,0.994,0.995,41916.0
MQTT-DDoS-Publish_Flood,0.992,0.990,0.150,0.158,0.261,0.272,8416.0
MQTT-DoS-Connect_Flood,0.982,0.982,0.893,0.895,0.935,0.936,3131.0
MQTT-DoS-Publish_Flood,0.543,0.546,0.999,0.999,0.704,0.706,8505.0
MQTT-Malformed_Data,0.861,0.879,0.283,0.296,0.426,0.443,1747.0
Recon-OS_Scan,0.561,0.641,0.320,0.307,0.408,0.415,3834.0
Recon-Ping_Sweep,0.800,0.912,0.172,0.167,0.283,0.282,186.0


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After)
Average Type,,,,,,
macro avg,0.800,0.811,0.696,0.697,0.703,0.705
weighted avg,0.899,0.899,0.900,0.900,0.896,0.896


,Accuracy (Before),Accuracy (After)
Accuracy,0.900,0.900


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After),Support
Label,,,,,,,
0,0.922,0.921,0.878,0.874,0.900,0.897,37607.0
1,0.997,0.997,0.998,0.998,0.998,0.998,1576575.0


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After)
Average Type,,,,,,
macro avg,0.960,0.959,0.938,0.936,0.949,0.947
weighted avg,0.995,0.995,0.995,0.995,0.995,0.995


,Accuracy (Before),Accuracy (After)
Accuracy,0.995,0.995


### Gradient Boosting Classifier Training

In [10]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# train gradient boosting classifier model
os.environ["LOKY_MAX_CPU_COUNT"] = "16"

# train histogram gradient boosting classifier model
hgbc_multi = HistGradientBoostingClassifier(random_state=42)
hgbc_binary = HistGradientBoostingClassifier(random_state=42)
hgbc_multi.fit(X_train_pca, y_train_multi)
hgbc_binary.fit(X_train_pca, y_train_binary)

c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 556, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1038

HistGradientBoostingClassifier(random_state=42)

#### Accuracy of Gradient Boosting Classifier Model

In [12]:
# check accuracy of gradient boosting classifier model
hgbc_y_pred_multi = hgbc_multi.predict(X_test_pca)
hgbc_y_pred_binary = hgbc_binary.predict(X_test_pca)

# classification reports
hgbc_multi_classification_report = classification_report(
    y_test_multi, hgbc_y_pred_multi, output_dict=True
)
hgbc_binary_classification_report = classification_report(
    y_test_binary, hgbc_y_pred_binary, output_dict=True
)
print(
    "\nClassification Report for RandomizedSearchCV Multi Class Classification:\n",
    hgbc_multi_classification_report,
)
print(
    "\nClassification Report for RandomizedSearchCV Binary Classification:\n",
    hgbc_binary_classification_report,
)


Classification Report for RandomizedSearchCV Multi Class Classification:
 {'ARP_Spoofing': {'precision': 0.09465195246179967, 'recall': 0.25573394495412843, 'f1-score': 0.13816604708798016, 'support': 1744.0}, 'Benign': {'precision': 0.8176036999566412, 'recall': 0.6016964926742362, 'f1-score': 0.6932279460196376, 'support': 37607.0}, 'MQTT-DDoS-Connect_Flood': {'precision': 0.9526265186609562, 'recall': 0.780060120240481, 'f1-score': 0.8577499704875458, 'support': 41916.0}, 'MQTT-DDoS-Publish_Flood': {'precision': 0.26601423487544484, 'recall': 0.10658269961977186, 'f1-score': 0.1521886664404479, 'support': 8416.0}, 'MQTT-DoS-Connect_Flood': {'precision': 0.4190439475713184, 'recall': 0.6943468540402428, 'f1-score': 0.522658973434307, 'support': 3131.0}, 'MQTT-DoS-Publish_Flood': {'precision': 0.5049118751805837, 'recall': 0.8218694885361552, 'f1-score': 0.6255313436842812, 'support': 8505.0}, 'MQTT-Malformed_Data': {'precision': 0.009780597409463389, 'recall': 0.04235832856325129, '

#### Optimizing Gradient Boosting Classifier Model

In [13]:
# create params grid
hgbc_params_grid = {
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.4],
    "max_iter": [100, 200, 400, 800, 1000],
    "max_leaf_nodes": [None, 10, 20, 40, 80, 100],
    "max_depth": [None, 10, 20, 40, 80, 100],
    "min_samples_leaf": [1, 10, 20, 40, 80, 100, 150, 200],
    "l2_regularization": [0.0, 0.1, 0.2, 0.4, 0.8, 1.0],
    "max_features": [0.0, 0.1, 0.2, 0.4, 0.8, 1.0],
    "max_bins": [50, 100, 200, 255],
}

# initialize RandomizedSearchCV object
hgbc_multi_randomized_search = RandomizedSearchCV(
    estimator=hgbc_multi,
    param_distributions=hgbc_params_grid,
    n_iter=20,
    random_state=42,
    n_jobs=-1,
    verbose=2,
    scoring="f1_macro",
)
hgbc_binary_randomized_search = RandomizedSearchCV(
    estimator=hgbc_binary,
    param_distributions=hgbc_params_grid,
    n_iter=20,
    random_state=42,
    n_jobs=-1,
    verbose=2,
    scoring="f1",
)
# start search on hyper parameters
hgbc_multi_search = hgbc_multi_randomized_search.fit(X_train_pca, y_train_multi)
hgbc_binary_search = hgbc_binary_randomized_search.fit(X_train_pca, y_train_binary)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\U

Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\U

In [14]:
# print best estimator and best parameters for both multi class and binary classifications
print(
    f"Best estimator for multi class classification: {hgbc_multi_search.best_estimator_}"
)
print(
    f"Best parameters for multi class classification: {hgbc_multi_search.best_params_}"
)
print(f"Best estimator for binary classification: {hgbc_binary_search.best_estimator_}")
print(f"Best parameters for binary classification: {hgbc_binary_search.best_params_}")

# print classification report for those parameters
hgbc_multi_y_pred_random_search = hgbc_multi_search.predict(X_test_pca)
hgbc_binary_y_pred_random_search = hgbc_binary_search.predict(X_test_pca)

# classification reports
hgbc_multi_search_classification_report = classification_report(
    y_test_multi, hgbc_multi_y_pred_random_search, output_dict=True
)
hgbc_binary_search_classification_report = classification_report(
    y_test_binary, hgbc_binary_y_pred_random_search, output_dict=True
)

print(
    "\nClassification Report for best parameters for multi class classification:\n",
    hgbc_multi_search_classification_report,
)
print(
    "\nClassification Report for best parameters for binary classification:\n",
    hgbc_binary_search_classification_report,
)

Best estimator for multi class classification: HistGradientBoostingClassifier(l2_regularization=1.0, learning_rate=0.2,
                               max_depth=80, max_features=0.2, max_iter=200,
                               max_leaf_nodes=None, random_state=42)
Best parameters for multi class classification: {'min_samples_leaf': 20, 'max_leaf_nodes': None, 'max_iter': 200, 'max_features': 0.2, 'max_depth': 80, 'max_bins': 255, 'learning_rate': 0.2, 'l2_regularization': 1.0}
Best estimator for binary classification: HistGradientBoostingClassifier(learning_rate=0.2, max_bins=200,
                               max_features=0.8, max_leaf_nodes=80,
                               min_samples_leaf=150, random_state=42)
Best parameters for binary classification: {'min_samples_leaf': 150, 'max_leaf_nodes': 80, 'max_iter': 100, 'max_features': 0.8, 'max_depth': None, 'max_bins': 200, 'learning_rate': 0.2, 'l2_regularization': 0.0}

Classification Report for best parameters for multi class c

#### Metric Tables for Gradient Boosting Classifier Model

In [15]:
classification_report_tables(
    hgbc_multi_classification_report,
    hgbc_multi_search_classification_report,
    algorithm_name="Histogram Gradient Boosting Classifier",
    classification_type="Multiclass Classification",
)

classification_report_tables(
    hgbc_binary_classification_report,
    hgbc_binary_search_classification_report,
    algorithm_name="Histogram Gradient Boosting Classifier",
    classification_type="Binary Classification",
)


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After),Support
Label,,,,,,,
ARP_Spoofing,0.095,0.416,0.256,0.479,0.138,0.445,1744.0
Benign,0.818,0.893,0.602,0.942,0.693,0.917,37607.0
MQTT-DDoS-Connect_Flood,0.953,0.983,0.780,0.996,0.858,0.989,41916.0
MQTT-DDoS-Publish_Flood,0.266,0.801,0.107,0.129,0.152,0.222,8416.0
MQTT-DoS-Connect_Flood,0.419,0.902,0.694,0.859,0.523,0.880,3131.0
MQTT-DoS-Publish_Flood,0.505,0.542,0.822,0.992,0.626,0.701,8505.0
MQTT-Malformed_Data,0.010,0.689,0.042,0.204,0.016,0.315,1747.0
Recon-OS_Scan,0.043,0.361,0.053,0.081,0.048,0.133,3834.0
Recon-Ping_Sweep,0.000,0.842,0.000,0.172,0.000,0.286,186.0


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After)
Average Type,,,,,,
macro avg,0.566,0.775,0.555,0.683,0.547,0.686
weighted avg,0.849,0.913,0.845,0.914,0.840,0.911


,Accuracy (Before),Accuracy (After)
Accuracy,0.845,0.914


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After),Support
Label,,,,,,,
0,0.918,0.913,0.876,0.905,0.897,0.909,37607.0
1,0.997,0.998,0.998,0.998,0.998,0.998,1576575.0


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After)
Average Type,,,,,,
macro avg,0.958,0.955,0.937,0.951,0.947,0.953
weighted avg,0.995,0.996,0.995,0.996,0.995,0.996


,Accuracy (Before),Accuracy (After)
Accuracy,0.995,0.996


### Multi-layer Perceptron (MLP) Classifier Training

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [17]:
# train MLP model
mlpc_multi = MLPClassifier(random_state=42)
mlpc_binary = MLPClassifier(random_state=42)
mlpc_multi.fit(X_train_pca, y_train_multi)
mlpc_binary.fit(X_train_pca, y_train_binary)

MLPClassifier(random_state=42)

#### Accuracy of MLP Classifier Model

In [18]:
# check accuracy of MLP neural network
# check accuracy of gradient boosting classifier model
mlpc_y_pred_multi = rf_multi.predict(X_test_pca)
mlpc_y_pred_binary = rf_binary.predict(X_test_pca)

# classification reports
mlpc_multi_classification_report = classification_report(
    y_test_multi, mlpc_y_pred_multi, output_dict=True
)
mlpc_binary_classification_report = classification_report(
    y_test_binary, mlpc_y_pred_binary, output_dict=True
)

print(
    "\nClassification Report for Multi-Class Classification:\n",
    mlpc_multi_classification_report,
)
print(
    "\nClassification Report for Binary Classification:\n",
    mlpc_binary_classification_report,
)


Classification Report for Multi-Class Classification:
 {'ARP_Spoofing': {'precision': 0.3142016188373804, 'recall': 0.4896788990825688, 'f1-score': 0.3827879874495742, 'support': 1744.0}, 'Benign': {'precision': 0.8891977163461539, 'recall': 0.9442656952163161, 'f1-score': 0.9159047238306488, 'support': 37607.0}, 'MQTT-DDoS-Connect_Flood': {'precision': 0.9898075013006669, 'recall': 0.9985447084645481, 'f1-score': 0.9941569083869742, 'support': 41916.0}, 'MQTT-DDoS-Publish_Flood': {'precision': 0.9921568627450981, 'recall': 0.15030893536121673, 'f1-score': 0.26106696935300794, 'support': 8416.0}, 'MQTT-DoS-Connect_Flood': {'precision': 0.9817415730337079, 'recall': 0.8930054295752156, 'f1-score': 0.9352734570998494, 'support': 3131.0}, 'MQTT-DoS-Publish_Flood': {'precision': 0.5434490696336083, 'recall': 0.9992945326278659, 'f1-score': 0.7040258449304175, 'support': 8505.0}, 'MQTT-Malformed_Data': {'precision': 0.8606271777003485, 'recall': 0.28277046365197483, 'f1-score': 0.425678586

### Optimizing MLP Classifier Model

In [ ]:
# create params grid for MLP Classifier
mlpc_params_grid = {
    "hidden_layer_sizes": [(100,), (200,), (100, 100), (200, 200)],
    "activation": ["identity", "logistic", "tanh", "relu"],
    "solver": ["lbfgs", "sgd", "adam"],
    "alpha": [0.0001, 0.001, 0.01, 0.1],
    "learning_rate": ["constant", "invscaling", "adaptive"],
    "learning_rate_init": [0.001, 0.01, 0.1],
    "max_iter": [200, 400, 800, 1000],
}

# initialize RandomizedSearchCV object for both multi class and binary classification
mlpc_multi_randomized_search = RandomizedSearchCV(
    estimator=mlpc_multi,
    param_distributions=mlpc_params_grid,
    n_iter=20,
    random_state=42,
    n_jobs=-1,
    verbose=2,
    scoring="f1_macro",
)
mlpc_binary_randomized_search = RandomizedSearchCV(
    estimator=mlpc_binary,
    param_distributions=mlpc_params_grid,
    n_iter=20,
    random_state=42,
    n_jobs=-1,
    verbose=2,
    scoring="f1",
)
# start search on hyper parameters
mlpc_multi_search = mlpc_multi_randomized_search.fit(X_train_pca, y_train_multi)
mlpc_binary_search = mlpc_binary_randomized_search.fit(X_train_pca, y_train_binary)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\spark\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [25]:
# print best estimator and best parameters for both multi class and binary classifications
print(
    f"Best estimator for multi class classification: {mlpc_multi_search.best_estimator_}"
)
print(
    f"Best parameters for multi class classification: {mlpc_multi_search.best_params_}"
)
print(f"Best estimator for binary classification: {mlpc_binary_search.best_estimator_}")
print(f"Best parameters for binary classification: {mlpc_binary_search.best_params_}")

# print classification report for those parameters
mlpc_multi_y_pred_random_search = mlpc_multi_search.predict(X_test_pca)
mlpc_binary_y_pred_random_search = mlpc_binary_search.predict(X_test_pca)

# classification reports
mlpc_multi_search_classification_report = classification_report(
    y_test_multi, mlpc_y_pred_multi, output_dict=True
)
mlpc_binary_search_classification_report = classification_report(
    y_test_binary, mlpc_y_pred_binary, output_dict=True
)
print(
    "\nClassification Report for best parameters for multi class classification:\n",
    mlpc_multi_search_classification_report,
)
print(
    "\nClassification Report for best parameters for binary classification:\n",
    mlpc_binary_search_classification_report,
)

Best estimator for multi class classification: MLPClassifier(hidden_layer_sizes=(200,), learning_rate='invscaling',
              max_iter=800, random_state=42, solver='lbfgs')
Best parameters for multi class classification: {'solver': 'lbfgs', 'max_iter': 800, 'learning_rate_init': 0.001, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (200,), 'alpha': 0.0001, 'activation': 'relu'}
Best estimator for binary classification: MLPClassifier(activation='logistic', alpha=0.001, hidden_layer_sizes=(100, 100),
              learning_rate_init=0.01, max_iter=400, random_state=42,
              solver='lbfgs')
Best parameters for binary classification: {'solver': 'lbfgs', 'max_iter': 400, 'learning_rate_init': 0.01, 'learning_rate': 'constant', 'hidden_layer_sizes': (100, 100), 'alpha': 0.001, 'activation': 'logistic'}

Classification Report for best parameters for multi class classification:
 {'ARP_Spoofing': {'precision': 0.3142016188373804, 'recall': 0.4896788990825688, 'f1-score': 0.38

#### MLP Classifier Metric Tables

In [26]:
classification_report_tables(
    mlpc_multi_classification_report,
    mlpc_multi_search_classification_report,
    algorithm_name="Multi-layer Perceptron Classifier",
    classification_type="Multiclass Classification",
)

classification_report_tables(
    mlpc_binary_classification_report,
    mlpc_binary_search_classification_report,
    algorithm_name="Multi-layer Perceptron Classifier",
    classification_type="Binary Classification",
)

,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After),Support
Label,,,,,,,
ARP_Spoofing,0.314,0.314,0.490,0.490,0.383,0.383,1744.0
Benign,0.889,0.889,0.944,0.944,0.916,0.916,37607.0
MQTT-DDoS-Connect_Flood,0.990,0.990,0.999,0.999,0.994,0.994,41916.0
MQTT-DDoS-Publish_Flood,0.992,0.992,0.150,0.150,0.261,0.261,8416.0
MQTT-DoS-Connect_Flood,0.982,0.982,0.893,0.893,0.935,0.935,3131.0
MQTT-DoS-Publish_Flood,0.543,0.543,0.999,0.999,0.704,0.704,8505.0
MQTT-Malformed_Data,0.861,0.861,0.283,0.283,0.426,0.426,1747.0
Recon-OS_Scan,0.561,0.561,0.320,0.320,0.408,0.408,3834.0
Recon-Ping_Sweep,0.800,0.800,0.172,0.172,0.283,0.283,186.0


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After)
Average Type,,,,,,
macro avg,0.800,0.800,0.696,0.696,0.703,0.703
weighted avg,0.899,0.899,0.900,0.900,0.896,0.896


,Accuracy (Before),Accuracy (After)
Accuracy,0.900,0.900


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After),Support
Label,,,,,,,
0,0.922,0.922,0.878,0.878,0.900,0.900,37607.0
1,0.997,0.997,0.998,0.998,0.998,0.998,1576575.0


,Precision (Before),Precision (After),Recall (Before),Recall (After),F1-score (Before),F1-score (After)
Average Type,,,,,,
macro avg,0.960,0.960,0.938,0.938,0.949,0.949
weighted avg,0.995,0.995,0.995,0.995,0.995,0.995


,Accuracy (Before),Accuracy (After)
Accuracy,0.995,0.995


### Comparing Algorithms

In [35]:
# extract both f1 scores from each algorithm
def extract_f1_scores(name, classification_type, report_before, report_after):
    return {
        "Algorithm": name,
        "Classification Type": classification_type,
        "Macro F1 (Before)": report_before["macro avg"]["f1-score"],
        "Macro F1 (After)": report_after["macro avg"]["f1-score"],
        "Weighted F1 (Before)": report_before["weighted avg"]["f1-score"],
        "Weighted F1 (After)": report_after["weighted avg"]["f1-score"],
    }


# get all f1 scores
f1_scores_full = [
    extract_f1_scores(
        "Random Forest Classifier",
        "Multiclass",
        rf_multi_classification_report,
        rf_multi_search_classification_report,
    ),
    extract_f1_scores(
        "Random Forest Classifier",
        "Binary",
        rf_binary_classification_report,
        rf_binary_search_classification_report,
    ),
    extract_f1_scores(
        "Histogram Gradient Boosting Classifier",
        "Multiclass",
        hgbc_multi_classification_report,
        hgbc_multi_search_classification_report,
    ),
    extract_f1_scores(
        "Histogram Gradient Boosting Classifier",
        "Binary",
        hgbc_binary_classification_report,
        hgbc_binary_search_classification_report,
    ),
    extract_f1_scores(
        "Multi-layer Perceptron Classifier",
        "Multiclass",
        mlpc_multi_classification_report,
        mlpc_multi_search_classification_report,
    ),
    extract_f1_scores(
        "Multi-layer Perceptron Classifier",
        "Binary",
        mlpc_binary_classification_report,
        mlpc_binary_search_classification_report,
    ),
]

# display the table
f1_df_full = pd.DataFrame(f1_scores_full)
styled_f1_df = (
    f1_df_full.style.set_caption(
        "Algorithm F1 Score Comparison (Before vs After Optimization)"
    )
    .format(
        {
            "Macro F1 (Before)": "{:.3f}",
            "Macro F1 (After)": "{:.3f}",
            "Weighted F1 (Before)": "{:.3f}",
            "Weighted F1 (After)": "{:.3f}",
        }
    )
    .background_gradient(
        cmap="YlGnBu",
        axis=None,
        subset=[
            "Macro F1 (Before)",
            "Macro F1 (After)",
            "Weighted F1 (Before)",
            "Weighted F1 (After)",
        ],
    )
    .hide(axis="index")
)

display(styled_f1_df)


Algorithm,Classification Type,Macro F1 (Before),Macro F1 (After),Weighted F1 (Before),Weighted F1 (After)
Random Forest Classifier,Multiclass,0.703,0.705,0.896,0.896
Random Forest Classifier,Binary,0.949,0.947,0.995,0.995
Histogram Gradient Boosting Classifier,Multiclass,0.547,0.686,0.840,0.911
Histogram Gradient Boosting Classifier,Binary,0.947,0.953,0.995,0.996
Multi-layer Perceptron Classifier,Multiclass,0.703,0.703,0.896,0.896
Multi-layer Perceptron Classifier,Binary,0.949,0.949,0.995,0.995


In [34]:
import json

# parameter comparison function (returns single row with full param sets as JSON)
def get_param_comparison_json(
    algorithm_name, classification_type, model_before, best_params_after
):
    before_params = model_before.get_params()
    return {
        "Algorithm": algorithm_name,
        "Classification Type": classification_type,
        "Parameters (Before)": json.dumps(before_params, indent=2),
        "Parameters (After)": json.dumps(best_params_after, indent=2),
    }

# list of parameter summaries
param_comparisons_json = []

# rf parameters
param_comparisons_json.append(
    get_param_comparison_json(
        "Random Forest", "Multiclass", rf_multi, rf_multi_search.best_params_
    )
)
param_comparisons_json.append(
    get_param_comparison_json(
        "Random Forest", "Binary", rf_binary, rf_binary_search.best_params_
    )
)

# hgbc parameters
param_comparisons_json.append(
    get_param_comparison_json(
        "Histogram Gradient Boosting",
        "Multiclass",
        hgbc_multi,
        hgbc_multi_search.best_params_,
    )
)
param_comparisons_json.append(
    get_param_comparison_json(
        "Histogram Gradient Boosting",
        "Binary",
        hgbc_binary,
        hgbc_binary_search.best_params_,
    )
)

# mlpc parameters
param_comparisons_json.append(
    get_param_comparison_json(
        "Multi-layer Perceptron",
        "Multiclass",
        mlpc_multi,
        mlpc_multi_search.best_params_,
    )
)
param_comparisons_json.append(
    get_param_comparison_json(
        "Multi-layer Perceptron", "Binary", mlpc_binary, mlpc_binary_search.best_params_
    )
)

# create and style DataFrame
param_df_json = pd.DataFrame(param_comparisons_json)

styled_param_df_json = param_df_json.style.set_caption(
    "Parameters (Before vs After)"
).hide(axis="index")

display(styled_param_df_json)


Algorithm,Classification Type,Parameters (Before),Parameters (After)
Random Forest,Multiclass,"{ ""bootstrap"": true, ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": ""sqrt"", ""max_leaf_nodes"": null, ""max_samples"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""monotonic_cst"": null, ""n_estimators"": 100, ""n_jobs"": -1, ""oob_score"": false, ""random_state"": 42, ""verbose"": 0, ""warm_start"": false }","{ ""n_estimators"": 100, ""min_samples_split"": 4, ""min_samples_leaf"": 1, ""max_features"": ""log2"", ""max_depth"": null, ""criterion"": ""gini"" }"
Random Forest,Binary,"{ ""bootstrap"": true, ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": ""sqrt"", ""max_leaf_nodes"": null, ""max_samples"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""monotonic_cst"": null, ""n_estimators"": 100, ""n_jobs"": -1, ""oob_score"": false, ""random_state"": 42, ""verbose"": 0, ""warm_start"": false }","{ ""n_estimators"": 1000, ""min_samples_split"": 10, ""min_samples_leaf"": 4, ""max_features"": ""sqrt"", ""max_depth"": 40, ""criterion"": ""entropy"" }"
Histogram Gradient Boosting,Multiclass,"{ ""categorical_features"": ""from_dtype"", ""class_weight"": null, ""early_stopping"": ""auto"", ""interaction_cst"": null, ""l2_regularization"": 0.0, ""learning_rate"": 0.1, ""loss"": ""log_loss"", ""max_bins"": 255, ""max_depth"": null, ""max_features"": 1.0, ""max_iter"": 100, ""max_leaf_nodes"": 31, ""min_samples_leaf"": 20, ""monotonic_cst"": null, ""n_iter_no_change"": 10, ""random_state"": 42, ""scoring"": ""loss"", ""tol"": 1e-07, ""validation_fraction"": 0.1, ""verbose"": 0, ""warm_start"": false }","{ ""min_samples_leaf"": 20, ""max_leaf_nodes"": null, ""max_iter"": 200, ""max_features"": 0.2, ""max_depth"": 80, ""max_bins"": 255, ""learning_rate"": 0.2, ""l2_regularization"": 1.0 }"
Histogram Gradient Boosting,Binary,"{ ""categorical_features"": ""from_dtype"", ""class_weight"": null, ""early_stopping"": ""auto"", ""interaction_cst"": null, ""l2_regularization"": 0.0, ""learning_rate"": 0.1, ""loss"": ""log_loss"", ""max_bins"": 255, ""max_depth"": null, ""max_features"": 1.0, ""max_iter"": 100, ""max_leaf_nodes"": 31, ""min_samples_leaf"": 20, ""monotonic_cst"": null, ""n_iter_no_change"": 10, ""random_state"": 42, ""scoring"": ""loss"", ""tol"": 1e-07, ""validation_fraction"": 0.1, ""verbose"": 0, ""warm_start"": false }","{ ""min_samples_leaf"": 150, ""max_leaf_nodes"": 80, ""max_iter"": 100, ""max_features"": 0.8, ""max_depth"": null, ""max_bins"": 200, ""learning_rate"": 0.2, ""l2_regularization"": 0.0 }"
Multi-layer Perceptron,Multiclass,"{ ""activation"": ""relu"", ""alpha"": 0.0001, ""batch_size"": ""auto"", ""beta_1"": 0.9, ""beta_2"": 0.999, ""early_stopping"": false, ""epsilon"": 1e-08, ""hidden_layer_sizes"": [ 100 ], ""learning_rate"": ""constant"", ""learning_rate_init"": 0.001, ""max_fun"": 15000, ""max_iter"": 200, ""momentum"": 0.9, ""n_iter_no_change"": 10, ""nesterovs_momentum"": true, ""power_t"": 0.5, ""random_state"": 42, ""shuffle"": true, ""solver"": ""adam"", ""tol"": 0.0001, ""validation_fraction"": 0.1, ""verbose"": false, ""warm_start"": false }","{ ""solver"": ""lbfgs"", ""max_iter"": 800, ""learning_rate_init"": 0.001, ""learning_rate"": ""invscaling"", ""hidden_layer_sizes"": [ 200 ], ""alpha"": 0.0001, ""activation"": ""relu"" }"
Multi-layer Perceptron,Binary,"{ ""activation"": ""relu"", ""alpha"": 0.0001, ""batch_size"": ""auto"", ""beta_1"": 0.9, ""beta_2"": 0.999, ""early_stopping"": false, ""epsilon"": 1e-08, ""hidden_layer_sizes"": [ 100 ], ""learning_rate"": ""constant"", ""learning_rate_init"": 0.001, ""max_fun"": 15000, ""max_iter"": 200, ""momentum"": 0.9, ""n_iter_no_change"": 10, ""nesterovs_momentum"": true, ""power_t